In [414]:
import numpy as np
import torch.optim
from scipy.optimize import minimize, least_squares, LinearConstraint, NonlinearConstraint
import torch.nn as nn
from torch.autograd import Variable
from enum import Enum
from torch import float64
from numdifftools import Jacobian, Hessian

In [415]:
Methods = Enum('Methods', ['Classic', 'Momentum', 'AdaGrad', 'RMSprop', 'Adam', 'Nesterov'])


def optimizer_handler(method, params, lr, beta_1=0.9, beta_2=0.999):
    match method:
        case Methods.Classic:
            return torch.optim.SGD(params, lr)
        case Methods.Momentum:
            return torch.optim.SGD(params, lr, beta_1)
        case Methods.AdaGrad:
            return torch.optim.Adagrad(params, lr)
        case Methods.RMSprop:
            return torch.optim.RMSprop(params, lr, alpha=beta_2)
        case Methods.Adam:
            return torch.optim.Adam(params, lr, betas=(beta_1, beta_2))
        case Methods.Nesterov:
            return torch.optim.SGD(params, lr, nesterov=True, momentum=beta_1)


class TorchLinearRegression:
    def __init__(self, T, X, Y):
        self.T = torch.zeros(len(X), len(T))
        for i in range(len(X)):
            for j in range(len(T)):
                self.T.data[i, j] = T[j](X.data[i])
        self.X = X
        self.Y = Y
        self.W = Variable(torch.randn(len(T), 1), requires_grad=True)

    def optimize(self, method=Methods.Classic, loss=nn.MSELoss(), lr=0.01, max_steps=1500):
        optimizer = optimizer_handler(method, [self.W], lr)

        for i in range(max_steps):
            optimizer.zero_grad()
            model_i = self.T.mm(self.W)
            loss_i = loss(model_i, self.Y)
            loss_i.backward()
            optimizer.step()

        return self.W

In [416]:
#Example 3

#Excepted
excepted = np.array([-2.34, 8.987, 118.12, 103.1])
M = len(excepted)

#Parameters
N = 100
deviation = 0.01
noise = torch.randn(N, 1) * deviation
powers = [(M - 1 - i) for i in range(M)]
Funcs = np.array([lambda x, i=i: (x ** powers[i]) for i in range(M)])
X1 = torch.randn(N, 1)
Y1 = sum([excepted[i] * Funcs[i](X1) for i in range(M)]) + noise

#Calculations
lin_reg = TorchLinearRegression(Funcs, X1, Y1)
received = lin_reg.optimize().detach().numpy().reshape(1, len(Funcs))[0]
print("Excepted: " + str(excepted))
print("Received: " + str(received))
print("Absolute error: " + str(np.linalg.norm(excepted - received)))
print("Relative error: " + str(np.linalg.norm(excepted - received) / np.linalg.norm(excepted)))

Excepted: [ -2.34    8.987 118.12  103.1  ]
Received: [ -2.3387651   8.986704  118.113815  103.1      ]
Absolute error: 0.006313713809522713
Relative error: 4.0199095694993615e-05


In [417]:
Algorithms = Enum('Methods', ['Newton', 'DogLeg', 'BFGS', 'LBFGS'])


def optimize_handler(fun, x0, algorithm=Algorithms.Newton):
    def fun_jac(x):
        return Jacobian(lambda x: fun(x))(x).ravel()

    def fun_hess(x):
        return Hessian(lambda x: fun(x))(x)

    match algorithm:
        case Algorithms.Newton:
            return least_squares(fun, x0)
        case Algorithms.DogLeg:
            return minimize(fun, x0, method='dogleg', jac=fun_jac, hess=fun_hess)
        case Algorithms.BFGS:
            return minimize(fun, x0, method='BFGS')
        case Algorithms.LBFGS:
            return minimize(fun, x0, method='L-BFGS-B')

In [418]:
#Example 4

#Excepted
excepted = np.array([-2.34, 8.987, 118.12, 103.1])
M = len(excepted)

#Parameters
N = 100
deviation = 0.01
noise = torch.randn(N, 1) * deviation
powers = [(M - 1 - i) for i in range(M)]
Funcs = np.array([lambda x, i=i: (x ** powers[i]) for i in range(M)])
X1 = torch.randn(N, 1)
Y1 = sum([excepted[i] * Funcs[i](X1) for i in range(M)]) + noise


#Function
def f1(W):
    W1 = np.copy(W).reshape(len(Funcs), 1)
    W1 = torch.tensor(W1, dtype=float64)
    T = torch.zeros(len(X1), len(Funcs), dtype=float64)
    for i in range(len(X1)):
        for j in range(len(Funcs)):
            T.data[i, j] = Funcs[j](X1.data[i])
    model = T.mm(W1)
    mse = nn.MSELoss()
    return mse(model, Y1).item()


#Calculations
x0 = torch.randn(M).detach().numpy().reshape(1, len(Funcs))[0]
received = optimize_handler(f1, x0, Algorithms.DogLeg).x
print("Excepted: " + str(excepted))
print("Received: " + str(received))
print("Absolute error: " + str(np.linalg.norm(excepted - received)))
print("Relative error: " + str(np.linalg.norm(excepted - received) / np.linalg.norm(excepted)))

Excepted: [ -2.34    8.987 118.12  103.1  ]
Received: [ -2.33967311   8.98772569 118.12032457 103.09883301]
Absolute error: 0.0014493753585504343
Relative error: 9.228099418833025e-06
